In [1]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = "1 2 3 4 5 6 7 8 9 1 2 3 4 5 6 7 8 9 1 2 3 4 5 6 7 8 9 1 2 3 4 5 6 7 8 9"
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 4 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while smooth_loss > 0.001:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if (n % 100 == 0): 
        print('iter %d, loss: %f' % (n, smooth_loss))# print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

data has 71 characters, 10 unique.
----
 3167162657439856413341178772956963691716349248694663525324463 7816548828884751382 126662477  895 648752137841939257 1724476765299288659498831314755438658719 2666 1298248  2954769152488 392317849 848 8 
----
iter 0, loss: 9.210340
----
  4 3 4 1 4 4 6 9 2 4 9 4 3 3 4 6 4 3 3 4 3 4 8 5 7 2 4 4 4 4 4 1 9 6 2 8 4 3 6 2 3 3 4 2 2 2 9 2 4 3 1 3 2 8 4 5 2 8 4 3 2 5 1 4 2 3 4 1 3 6 9 4 4 4 3 8 8 9 1 3 2 8 2 4 3 9 4 4 5 3 7 4 2 2 2 3 3 4 8 9 
----
iter 100, loss: 9.091372
----
  6 4 4 8 4 8 8 9 4 7 6 6 3 9 3 3 7 8 3 9 6 9 9 8 9 6 4 6 5 3 6 3 5 9 5 8 4 4 9 8 2 8 5 9 8 9 8 2 9 9 9 8 8 5 7 9 1 7 6 3 9 6 4 7 6 3 6 6 9 8 4 8 2 4 5 9 4 7 8 6 7 6 9 9 6 8 7 8 9 6 5 3 8 9 3 8 3 8 7 8 
----
iter 200, loss: 8.752370
----
  4 4 4 7 2 1 9 2 9 9 6 8 1 5 7 2 2 3 1 9 4 9 1 5 5 4 5 3 2 4 5 5 5 7 1 5 2 4 7 7 4 2 4 2 5 3 2 4 9 1 3 9 5 1 5 3 3 3 2 4 3 5 7 2 1 5 2 5 4 4 3 5 7 1 3 8 5 5 4 9 5 6 5 4 5 3 7 9 5 2 2 4 5 3 5 6 7 2 4 9 
----
iter 300, loss: 8.406845
----
  9 9 1 4 

In [10]:
inDat = [char_to_ix[ch] for ch in "4"]
sample(np.zeros((hidden_size,1)), inDat, 100)

[8,
 4,
 8,
 2,
 8,
 3,
 8,
 2,
 8,
 5,
 8,
 9,
 8,
 0,
 8,
 1,
 8,
 6,
 8,
 4,
 8,
 7,
 8,
 3,
 8,
 2,
 8,
 5,
 8,
 9,
 8,
 0,
 8,
 1,
 8,
 6,
 8,
 4,
 8,
 7,
 8,
 3,
 8,
 2,
 8,
 5,
 8,
 9,
 8,
 0,
 8,
 1,
 8,
 6,
 8,
 4,
 8,
 7,
 8,
 3,
 8,
 2,
 8,
 5,
 8,
 9,
 8,
 0,
 8,
 1,
 8,
 6,
 8,
 4,
 8,
 7,
 8,
 3,
 8,
 2,
 8,
 5,
 8,
 9,
 8,
 0,
 8,
 1,
 8,
 6,
 8,
 4,
 8,
 7,
 8,
 3,
 8,
 2,
 8,
 5]